We are trying to determine the neutral point, or $x_{np}$. We determine it through the following equation:

$$
\bar{x}_{\text{np}} = \frac{\sum_{i=1}^{3} l_{h_i} S_{h_i}}{\bar{c} S_w} 
\left( \frac{\sum_{i=1}^{3} \frac{\partial C_{L_{h_i}}}{\partial \alpha} S_{h_i}}{\partial C_{L_w} / \partial \alpha} \right) 
- \frac{\partial C_{m_{\text{fus}}}}{\partial \alpha} \left( \frac{\partial C_{L_w}}{\partial \alpha} \right)^{-1}
$$


### Where:
- $\bar{x}_{\text{np}}$ = Neutral point location (non-dimensionalized by wing mean aerodynamic chord)

- $l_h$ = Distance from the aerodynamic center of the wing to the aerodynamic center of the horizontal tail

- $S_h$ = Horizontal tail planform area

- $\bar{c}$ = Mean aerodynamic chord of the wing

- $S_w$ = Wing planform area

- $\frac{\partial C_{L_h}}{\partial \alpha}$ = Change in horizontal tail lift coefficient with respect to angle of attack

- $\frac{\partial C_{L_w}}{\partial \alpha}$ = Change in wing lift coefficient with respect to angle of attack

- $\frac{\partial C_{m_{\text{fus}}}}{\partial \alpha}$ = Change in fuselage pitching moment coefficient with respect to angle of attack


In [ ]:
import numpy as np

M = 0.302       #mach for 200 knots
eff = 0.97  #accounts for the difference between the theoretical section lift curve slope of 2 and the actual value

fus_w = 6 #ft
fus_l = 30 #ft

#MAIN WING GEOM
root_w = 8   #ft
tip_w = 4  #ft
b_w = 52  #ft
lambda_w = np.radians(10)

#CANARD GEOM
root_c = 8/3 #ft
tip_c = 8/3 #ft
b_c = 20 #ft
lamda_c = np.radians(0)

#HOR-STAB GEOM
root_hs = 2.5 #ft
tip_hs = 2.5 #ft
b_hs = 16 #ft
lamda_hs = np.radians(0)

x_CG = 0  

del_c = 23/3   #space between back of canard and front of wing (ft)
l_c = -(del_c + root_c * 0.75) - x_CG     #making sure its from the quarter chords

del_hs = 54/3  #space between front of wing and front of horizontal stabilizers (ft)
l_hs = (del_hs + root_hs * 0.25) - x_CG  #making sure its from the quarter chords

del_w = 15 + root_w * 0.25      #space between quarter chord of wing and front of fuselage
spot_w = del_w / fus_l

temp = (0.6 - spot_w)/(spot_w - 0.5)
K_f = (temp * 0.688 + 0.888) / (temp + 1)

def getS(root, tip, span):
    return (span * (root + tip) / 2)

def getAR(b, S):
    return ((b**2) / S)

def getdCL_da(AR, lam):
    return ((2 * np.pi * AR) / (2 + np.sqrt((AR / eff)**2 * (1 + (np.tan(lam)**2 - M**2) + 4))))


S_w = getS(root_w,tip_w,b_w)
S_c = getS(root_c,tip_c,b_c)
S_hs = getS(root_hs,tip_hs,b_hs)

AR_w = getAR(b_w, S_w)
AR_c = getAR(b_c, S_c)
AR_hs = getAR(b_hs, S_hs)

dCLw_da = getdCL_da(AR_w, lambda_w)
dClc_clean_da = getdCL_da(AR_c, lamda_c)
dClhs_clean_da = getdCL_da(AR_hs, lamda_hs)

de_da = 2 / (np.pi * AR_w) * dCLw_da

dClc_da = dClc_clean_da * (1 + de_da)
dClhs_da = dClhs_clean_da * (1 - de_da)

taper_ratio = tip_w / root_w
c_bar = 2/3 * root_w * (1 + taper_ratio + taper_ratio**2) / (1 + taper_ratio)

dCmfus_da = K_f  * fus_w**2 * fus_l / (S_w * c_bar)

factor_c = l_c * S_c / (c_bar * S_w) * dClc_da / dCLw_da
factor_hs = l_hs * S_hs / (c_bar * S_w) * dClhs_da / dCLw_da
factor_fus = dCmfus_da / dCLw_da

x_np_c = (-factor_c + 2 * factor_hs - factor_fus)
x_NP = x_np_c * c_bar
SM = x_NP - x_CG
SM_c = SM / c_bar

print(f"Neutral point is {x_np_c:.5f}c̄, or {x_NP:.5f} feet.")
print(f"Static Margin is {SM_c:.5f}c̄, or {SM:.5f} feet.")

6.222222222222221
Neutral point is 0.73629c̄, or 4.58136 feet.
Static Margin is 0.73629c̄, or 4.58136 feet.
